In [45]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import sorting
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    sorting,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from mathematics import prime_numbers as primes
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(edgeitems=5, precision=3, linewidth=300)

In [142]:
def merge_email_accounts(accounts: list) -> list:
    n = len(accounts)
    parents = list(range(n))

    def parent(n: int) -> int:
        """Find the parent of n. Compress the path."""
        while n != parents[n]:
            # Compress the set by halving.
            n, parents[n] = parents[n], parents[parents[n]]
        return n

    emails = {}
    for i, a in enumerate(accounts):
        for e in a[1:]:
            parents[i] = i = parent(emails.setdefault(e, i))  # noqa: PLW2901

    merged = [[] for _ in range(n)]
    for e, i in emails.items():
        merged[parent(i)].append(e)
    any(map(list.sort, merged))
    return sorted([accounts[i][0], *m] for i, m in enumerate(merged) if m)

In [143]:
merge_email_accounts(
    [
        ["John", "johnsmith@mail.com", "john_newyork@mail.com"],
        ["John", "johnsmith@mail.com", "john00@mail.com"],
        ["Mary", "mary@mail.com"],
        ["John", "johnnybravo@mail.com"],
    ]
)

[['John', 'john00@mail.com', 'john_newyork@mail.com', 'johnsmith@mail.com'],
 ['John', 'johnnybravo@mail.com'],
 ['Mary', 'mary@mail.com']]

In [139]:
merge_email_accounts(
    [
        ["Gabe", "Gabe00@m.co", "Gabe3@m.co", "Gabe1@m.co"],
        ["Kevin", "Kevin3@m.co", "Kevin5@m.co", "Kevin0@m.co"],
        ["Ethan", "Ethan5@m.co", "Ethan4@m.co", "Ethan0@m.co"],
        ["Hanzo", "Hanzo3@m.co", "Hanzo1@m.co", "Hanzo0@m.co"],
        ["Fern", "Fern5@m.co", "Fern1@m.co", "Fern0@m.co"],
    ]
)

[['Ethan', 'Ethan0@m.co', 'Ethan4@m.co', 'Ethan5@m.co'],
 ['Fern', 'Fern0@m.co', 'Fern1@m.co', 'Fern5@m.co'],
 ['Gabe', 'Gabe00@m.co', 'Gabe1@m.co', 'Gabe3@m.co'],
 ['Hanzo', 'Hanzo0@m.co', 'Hanzo1@m.co', 'Hanzo3@m.co'],
 ['Kevin', 'Kevin0@m.co', 'Kevin3@m.co', 'Kevin5@m.co']]